## Keypoint Description

### Import packages

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from Utils import L2Norm, cv2_scale, np_reshape
from descriptor import DesNet


### Check GPU availability, using nvidia-smi 

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<a id='load_weights_module_cell'></a>
### Load network weights
[BackToSection](#generate_deep_features_cell)

In [3]:
trained_weight_path = "checkpoint.pth" # suppose you are using checkpoint.pth
model = DesNet()
if args.cuda:
    model.cuda()
trained_weight = torch.load(trained_weight_path)['state_dict']
model.load_state_dict(trained_weight)
model.eval()

DesNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (14): Re

<a id='load_raw_patch_files_module_cell'></a>
### Load raw patch files
Assume that the raw patch file is stored as patches.pth

[BackToSection](#generate_deep_features_cell)

patches_image

In [4]:
patches_image_dir = "patches_image.pt"
patches_image = torch.load(patches_image_dir)
print(patches_image.shape)
patches_image = patches_image.view(-1, 1, 32, 32).cuda()
print(patches_image.shape)

torch.Size([136, 20, 1, 32, 32])
torch.Size([2720, 1, 32, 32])


patches_query

In [5]:
patches_query_dir = "patches_query.pt"
patches_query = torch.load(patches_query_dir)
print(patches_query.shape)
patches_query = patches_query.view(-1, 1, 32, 32).cuda()
print(patches_query.shape)

torch.Size([34, 20, 1, 32, 32])
torch.Size([680, 1, 32, 32])


<a id='get_deep_features_module_cell'></a>
### Get deep features
[BackToSection](#generate_deep_features_cell)

image

In [6]:
features_image = model(patches_image)
print(features_image.shape)
features_image = features_image.view(-1, 20, 128).cpu().data
print(features_image.shape)

torch.Size([2720, 128])
torch.Size([136, 20, 128])


In [7]:
# save to file, with the name of features_CNN*.pth
features_dir = "image_description.pth"
torch.save(features_image, features_dir)

query

In [8]:
features_query = model(patches_query)
print(features_query.shape)
features_query = features_query.view(-1, 20, 128).cpu().data
print(features_query.shape)

torch.Size([680, 128])
torch.Size([34, 20, 128])


In [9]:
# save to file, with the name of features_CNN*.pth
features_dir = "query_description.pth"
torch.save(features_query, features_dir)

### Stack query and image features

In [10]:
query_des = torch.load("query_description.pth")
image_des = torch.load("image_description.pth")

In [11]:
print(query_des.shape)
print(image_des.shape)

torch.Size([34, 20, 128])
torch.Size([136, 20, 128])


In [12]:
features = torch.cat([query_des, image_des], dim=0)
print(features.shape)
torch.save(features, "features.pth")

torch.Size([170, 20, 128])
